### Table for paper, Excess Return and Tracking Err for diff paramter values on 2 index files

In [1]:
base_path="../eit_basic_experiments/exp_3" #Don't add '/' at start and end
T=200 #Training time period
C=1000000 #Capital available
file=1 #Index_file
m=8 #Inital Size of Kernel
lbuck=2 #Size Of Buckets
pho=0.1 #Transaction Cost Proportion
lamda=1/(1000*C) # lower_bound for capital invested in jth stock
nuh=0.8  # upper_bound
xii=1.2 # Limit for Tracking Error
k=14 #Cardinality constraint for stocks in portfolio
p=3 #If stock not selected in optimal soln in last p iters then it is to be dropped from kernel
f=12
#cj,cb are declared in script

In [2]:
import sys
sys.path.insert(0,"..") #Add root folder to sys path

from eit_basic_experiment import TestEitBasic
import time
from tqdm.notebook import tqdm
from IPython.utils import io
from joblib import Parallel, delayed
import pandas as pd

In [3]:
!rm -r ../eit_basic_experiments/exp_3/*

### Loop over files > Loop over paramter values and publish table
1. In sample Excess return/Tracking Err
2. Out of Sample

In [4]:
path_pattern=base_path+"/file={},pho={},xii={},k={},m={},lbuck={},p={}"

#### Make runs for parallel process

In [16]:
%%time
runs=[]
for file in [1,8]:
    for pho in [0.1,0.2,0.5]:
        for xii in [1.2,1.3,1.4]:
            for k in [10,16,25]:
                output=path_pattern.format(file,pho,xii,k,m,lbuck,p)
                test_eit=TestEitBasic(output=output,T=T,C=C,file=file,m=m,lbuck=lbuck,\
                                        pho=pho,lamda=lamda,nuh=nuh,xii=xii,k=k,p=p,f=f)
                runs.append(test_eit)

CPU times: user 501 µs, sys: 0 ns, total: 501 µs
Wall time: 507 µs


In [6]:
def parralel_func(eit_object):
    params=eit_object.give_params()
    result_df=eit_object.run_experiment(from_root=False,verbose=False);
    result_df["z_linear"]=eit_object.objective_linear
    step_dict={}
    step_dict[params]=result_df
    return (step_dict)

In [7]:
result_dict_list={}
result_dicta.append(Parallel(verbose = 1,n_jobs  = -1\
            )(delayed( parralel_func )( run,result_dict) for run in tqdm(runs)))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 70.1min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 82.0min


TypeError: 'NoneType' object does not support item assignment

In [8]:
final_result=pd.DataFrame()
for params,result_df in result_dict.items():
    param_part=pd.DataFrame([params])
    res_part=pd.DataFrame([result_df.loc[result_df.z_value.idxmax()]])
    temp=pd.concat([param_part,res_part.reset_index()],axis=1,ignore_index=False)
    final_result=final_result.append(temp)

In [ ]:
final_result.to_csv(base_path+"/exp_3_results_table.csv")

In [9]:
final_result.groupby(["k","m"])[["z_value","z_linear"]].mean()

KeyError: 'k'

In [11]:
result_dict

{}